Hannah Stouter  
May 25, 2025  
##### Search, Request, and Download Sentinel-1 data from NASA Earth Data and the Alaska Satellite Facility  
This was put together based on documentation from: ASF_search: https://docs.asf.alaska.edu/  


In [106]:
import asf_search as asf
import hyp3_sdk as sdk
from hyp3_sdk import HyP3
import earthaccess
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box
from pathlib import Path
import pandas as pd
import os
import getpass


Set ROI for geographic search

In [105]:
# to get correct CRS
pyproj.datadir.set_data_dir("/Users/hstouter/anaconda3/envs/remote_sensing/share/proj")

shp_path = ("/Users/hstouter/Desktop/NASA_LCLUC/RF_datasets/ynd_test_buffer.shp")
shp = gpd.read_file(shp_path)
print(shp.head())

# Check CRS & update if needed
print("original :", shp.crs)

# Update CRS
shp = shp.to_crs(epsg=4326)
print("updated :", shp.crs)

# Get bounding box coordinates
minx, miny, maxx, maxy = shp.total_bounds

# Create a shapely box (polygon)
bbox = box(minx, miny, maxx, maxy)

# Fix geometry (ensure it's valid and CCW)
bbox_fixed = bbox.buffer(0)

# Convert to WKT
roi = bbox_fixed.wkt

print(roi)



   Id                                           geometry
0   0  POLYGON ((1277436.254 393886.351, 1253203.36 3...
original : EPSG:3857
updated : EPSG:4326
POLYGON ((11.910780699769553 3.5360943692198314, 11.257717326443535 3.5360943692198314, 11.257717326443535 4.1376852081059345, 11.910780699769553 4.1376852081059345, 11.910780699769553 3.5360943692198314))


Log into NASA Earthdata

In [64]:
def login_earthaccess():
    try:
        # Login to EarthAccess
        earthaccess.login(persist=True)
        
        # Get token information and store in header
        urs = 'urs.earthdata.nasa.gov'
        token_response = requests.post(
            f'{APPEEARS_API_ENDPOINT}login', 
            auth=(netrc().authenticators(urs)[0], 
                netrc().authenticators(urs)[2])
        ).json()
        token = token_response['token']
        head = {'Authorization': f'Bearer {token}'}
        
        return head
    
    except Exception as e:
        print(f'Error logging into EarthAccess. Error: {e}', flush=True)
        time.sleep(SLEEP_TIME)
        login_earthaccess() # Retry until we can login
        return None

Search for Sentinel-1 data

In [103]:
# Search scenes and get granule ids for RTC processing

results_1 = asf.geo_search(
    intersectsWith=roi,
    platform=asf.PLATFORM.SENTINEL1,
    processingLevel=asf.PRODUCT_TYPE.SLC,
    start='2021-10-01',
    end='2022-09-30'
)

results_2 = asf.geo_search(
    intersectsWith=roi,
    platform=asf.PLATFORM.SENTINEL1,
    processingLevel=asf.PRODUCT_TYPE.METADATA_GRD_HD,
    start='2021-10-01',
    end='2022-09-30'
)

results = results_1 + results_2

granule_ids = [result.properties['sceneName'] for result in results]
print("Granule IDs:")
for gid in granule_ids:
    print(gid)


["'type': 'REVERSE': 'report': Reversed polygon winding order"]


["'type': 'REVERSE': 'report': Reversed polygon winding order"]


S1A_IW_SLC__1SDV_20220926T173638_20220926T173706_045179_05665D_695E
S1A_IW_SLC__1SDV_20220921T172842_20220921T172910_045106_0563E3_09FF
S1A_IW_SLC__1SDV_20220921T172814_20220921T172844_045106_0563E3_FC70
S1A_IW_SLC__1SDV_20220914T173638_20220914T173706_045004_056077_C107
S1A_IW_SLC__1SDV_20220909T172843_20220909T172910_044931_055DF8_7A9D


Request a job for RTC processing

In [104]:
# Request a job for RTC processing
rtc_jobs = sdk.Batch()
for g in granule_ids:
    rtc_jobs += hyp3.submit_rtc_job(g, 
                                    name='RTC_sigma_test',
                                    radiometry='sigma0',
                                    dem_matching=True,
                                    scale='decibel',
                                    resolution=30,
                                    speckle_filter=True,
                                    include_rgb=True)
print(rtc_jobs)

5 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 5 pending.


Download request

In [96]:
# Set your output directory
outDir = ("/Users/hstouter/Library/CloudStorage/Box-Box/NASA - Land Cover Land Use Change (2023 - 2026)/Data/LCLUC/Sentinel_1/sigma_2022_test")
os.chdir(outDir)  # switch to that directory

# Check log in
hyp3 = sdk.HyP3()

# Search jobs by date
all_jobs = hyp3.find_jobs(
    start="2025-05-23T00:00:00Z",
    end="2025-05-23T23:59:59Z"
)

# View jobs and their status
print(all_jobs)

succeeded_jobs = all_jobs.filter_jobs(succeeded=True, running=False, failed=False)
print(f'Number of succeeded jobs: {len(succeeded_jobs)}')
failed_jobs = all_jobs.filter_jobs(succeeded=False, running=False, failed=True)
print(f'Number of failed jobs: {len(failed_jobs)}')

# Download jobs 
file_list = succeeded_jobs.download_files()







87 HyP3 Jobs: 87 succeeded, 0 failed, 0 running, 0 pending.
Number of succeeded jobs: 87
Number of failed jobs: 0


S1A_IW_20211008T172837_DVP_RTC30_G_sdufem_F200.zip: 100%|██████████| 554M/554M [00:57<00:00, 10.1MB/s]
S1A_IW_20211025T173632_DVP_RTC30_G_sdufem_3758.zip: 100%|██████████| 532M/532M [00:48<00:00, 11.4MB/s]
S1A_IW_20211020T172809_DVP_RTC30_G_sdufem_3A78.zip: 100%|██████████| 589M/589M [00:51<00:00, 12.0MB/s]
S1A_IW_20211001T173632_DVP_RTC30_G_sdufem_2327.zip: 100%|██████████| 533M/533M [00:28<00:00, 19.4MB/s]
S1A_IW_20211013T173632_DVP_RTC30_G_sdufem_D2A9.zip: 100%|██████████| 530M/530M [01:01<00:00, 9.01MB/s]
S1A_IW_20211008T172809_DVP_RTC30_G_sdufem_CAA4.zip: 100%|██████████| 588M/588M [00:46<00:00, 13.4MB/s]
S1A_IW_20211020T172837_DVP_RTC30_G_sdufem_5559.zip: 100%|██████████| 554M/554M [00:30<00:00, 19.1MB/s]
S1A_IW_20211106T173632_DVP_RTC30_G_sdufem_948E.zip: 100%|██████████| 532M/532M [00:31<00:00, 18.0MB/s]
S1A_IW_20211212T173631_DVP_RTC30_G_sdufem_E3FC.zip: 100%|██████████| 530M/530M [00:44<00:00, 12.5MB/s]
S1A_IW_20211219T172835_DVP_RTC30_G_sdufem_B9B9.zip: 100%|██████████| 555M